In [1]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse 
from tqdm import tqdm 

from models import * 

In [2]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print('Epoch: %d | Loss: %.3f | Acc: %.3f%% (%d/%d)'% 
              (epoch, train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [3]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() 

            print('Epoch: %d | Loss: %.3f | Acc: %.3f%% (%d/%d)'% 
                  ( epoch, test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [4]:
resume = False 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = ResNet18()
net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                    momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

for epoch in range(start_epoch, start_epoch+2):
    train(epoch)
    test(epoch)
    scheduler.step()


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..

Epoch: 0
Epoch: 0 | Loss: 2.347 | Acc: 12.500% (16/128)
Epoch: 0 | Loss: 2.323 | Acc: 12.891% (33/256)
Epoch: 0 | Loss: 2.312 | Acc: 11.719% (45/384)
Epoch: 0 | Loss: 2.302 | Acc: 12.891% (66/512)
Epoch: 0 | Loss: 2.280 | Acc: 14.844% (95/640)
Epoch: 0 | Loss: 2.270 | Acc: 15.365% (118/768)
Epoch: 0 | Loss: 2.264 | Acc: 15.848% (142/896)
Epoch: 0 | Loss: 2.260 | Acc: 16.211% (166/1024)
Epoch: 0 | Loss: 2.243 | Acc: 16.840% (194/1152)
Epoch: 0 | Loss: 2.227 | Acc: 17.344% (222/1280)
Epoch: 0 | Loss: 2.217 | Acc: 17.969% (253/1408)
Epoch: 0 | Loss: 2.206 | Acc: 18.294% (281/1536)
Epoch: 0 | Loss: 2.197 | Acc: 18.810% (313/1664)
Epoch: 0 | Loss: 2.181 | Acc: 19.364% (347/1792)
Epoch: 0 | Loss: 2.172 | Acc: 19.740% (379/1920)
Epoch: 0 | Loss: 2.167 | Acc: 20.068% (411/2048)
Epoch: 0 | Loss: 2.155 | Acc: 20.496% (446/2176)
Epoch: 0 | Loss: 2.151 | Acc: 20.833% (480/2304)
Ep

KeyboardInterrupt: 